<a href="https://colab.research.google.com/github/huangtinglin/test_colab/blob/main/CPSC483_colab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## GNN for link prediction and graph classification task

Last time we explore a standard benchmark datase Cora, and implement a classic graph neural network GCN(Kipf et al. (2017)) for node classification task. In this Colab, we are going to explore two kinds of graph learning task: **link prediction** and **graph classification**. We will apply GCN to the link prediction task, and implement a novel GNN model [GIN](https://arxiv.org/abs/1810.00826) for graph classification task. All of these implementations are still based on the [PyG](https://pytorch-geometric.readthedocs.io/en/latest/).


## Outline



- Link prediction task
- Graph classification task

In [1]:
# import the pytorch library into environment and check its version
import os
import torch
print("Using torch", torch.__version__)

Using torch 1.12.1+cu113


Let's start installing PyG by `pip`. The version of PyG should match the current version of PyTorch. Here we follow the [instruction](https://pytorch-geometric.readthedocs.io/en/latest/notes/installation.html) of PyG:

In [2]:
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.12.0+cu113.html
!pip install ogb  # for datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.12.0+cu113.html
     |████████████████████████████████| 7.9 MB 7.9 MB/s 
     |████████████████████████████████| 3.5 MB 63.9 MB/s 
     |████████████████████████████████| 2.4 MB 70.9 MB/s 
     |████████████████████████████████| 709 kB 79.2 MB/s 
     |████████████████████████████████| 467 kB 8.0 MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.1.0.post1-py3-none-any.whl size=689859 sha256=f14f39f0929f79a8a67613d505ef276d6f7003197a7e07dd194ed39891c816c3
  Stored in directory: /root/.cache/pip/wheels/d1/cb/43/f7f2e472de4d7cff31bceddadc36d634e1e545fbc17961c282
Successfully built torch-geometric
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 78 kB 228 kB/s 
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.w

Import some required libraries into our environment:

In [10]:
from torch_geometric.data import Data
from torch_geometric.datasets import Planetoid
from torch_geometric import nn
import torch_geometric.transforms as T



## Link prediction task


### Dataset preprocess

As shown in the following figure, link prediction is to predict whether two nodes in a graph have a link, which can be considered as a binary classification task. We will construct a link prediction dataset containing training, validation, and test set based on Cora. 


<br/>
<center>
<img src="https://github.com/Graph-and-Geometric-Learning/CPSC483-colab/blob/main/fig/link_prediction_example.png?raw=1" height="200" width="200"/>
</center>
<br/>


Given a graph, we divide the initial edge set into three distinct edge sets which represent the training, validation, and test set. Training set and validation set share a same graph structure. Test set contains some edges which does not exist in training and validation set to prevent data leakage.
<!-- Training set does not include edges in validation and test set, and the validation split does not include edges in the test split. Validation and test data should not be leaked into the training set. -->


<br/>
<center>
<img src="https://github.com/Graph-and-Geometric-Learning/CPSC483-colab/blob/main/fig/link_prediction_dataset_split(2).png?raw=1" height="200" width="350"/>
</center>
<br/>


Our model will be optimized on the training set. We can use `transforms` function in PyG to easily generate the data splits:

In [113]:
transform = T.Compose([
    T.RandomLinkSplit(num_val=0.05,  # ratio of edges including in the validation set
                      num_test=0.2,  # ratio of edges including in the test set
                      is_undirected=True,
                      add_negative_train_samples=False),
])

Loading the Cora dataset:

In [114]:
dataset = Planetoid('/tmp/cora', 'cora', transform=transform)

The data will be transformed from a data object to three tuples, where each element represents the corresponding split:

In [115]:
train_data, val_data, test_data = dataset[0]

Now data object has two attributes of edge: `edge_index` and `edge_label_index`. `edge_index` denotes the graph structure used for performing message passing in GNN. `edge_label_index` denotes the edge index used to calculate loss in training set, or to evaluate the model in validation and test set.


Printing the statistics of data:

In [116]:
print("Number of the nodes in training, validation and test data are", train_data.num_nodes, val_data.num_nodes, test_data.num_nodes)
print("Number of the edges in training, validation and test data are", train_data.num_edges, val_data.num_edges, test_data.num_edges)
print("Number of the edge_label_index in training, validation and test data are", train_data.edge_label_index.shape[1], 
                                                                                  val_data.edge_label_index.shape[1],
                                                                                  test_data.edge_label_index.shape[1])

Number of the nodes in training, validation and test data are 2708 2708 2708
Number of the edges in training, validation and test data are 7920 7920 8446
Number of the edge_label_index in training, validation and test data are 3960 526 2110


### Pipeline

We constructed the GCN by PyG in the last Colab, and now we simply use the same architecture:

In [93]:
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()

        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)
        self.act = torch.nn.ReLU()

    def forward(self, node_feature, edge_index):

        output = self.conv1(node_feature, edge_index)
        output = self.act(output)
        output = self.conv2(output, edge_index)

        return output

Initializing a GCN model:

In [127]:
model = GCN(dataset.num_features, hidden_channels=128, out_channels=64)

Define a optimizer for model:

In [128]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)

Similar as the what we do in the node classification task, we first apply the GCN model to produce the representation of each node in the graph. Usually we will use **inner product** to measure the similarity between two node representations to determine how likely it is for these two nodes to be connected.

#### Question 1

Following the instruction and implement the function to calculate the inner product:

In [88]:
def compute_similarity(node_embs, edge_index):
    result = 0

    # TODO: Define similarity function.
    # 1. calculate the inner product between all the pairs in the edge_index
    # Note: the shape of node_embs is [n, h] where n is the number of nodes, and h is the embedding size
    # the shape of edge_index is [2, m] where m is the number of edges

    ############# Your code here ############
    ## (~1 line of code)
    result = (node_embs[edge_index[0]] * node_embs[edge_index[1]]).sum(dim=-1)

    #########################################

    return result

n, h = 5, 10  # number of nodes and embedding size
node_embs = torch.rand(n, h)
edge_index = torch.tensor([[0, 1, 2, 3], 
                           [2, 3, 0, 1]])  # compute the similarity of (0, 2), (1, 3), (2, 0), (3, 1)
similarity = compute_similarity(node_embs, edge_index)
print("Similairty:", similarity)

Similairty: tensor([3.2714, 2.8681, 3.2714, 2.8681])


We optimize the model by minimizing the loss function. Here we consider the link prediction task as a binary classification task (edge exists or no), and apply binary cross entropy loss:

In [99]:
loss_fn = torch.nn.BCEWithLogitsLoss()

The edges in the graph will be taken as the positive examples with label=1 in the loss function. To prevent model from collapse, we usually will feed some **negative examples** to the loss function, which is the non-existing edges in the graph. The number of negative examples should equal to the number of positive ones.

With the help of PyG, we can easily perform the negative sampling. Here is an example:

In [97]:
from torch_geometric.utils import negative_sampling

neg_edge_index = negative_sampling(
      edge_index=train_data.edge_index,  # positive edges in the graph
      num_nodes=train_data.num_nodes,  # number of nodes
      num_neg_samples=5,  # number of negative examples
    )

print("shape of neg_edge_index:", neg_edge_index.shape)  # [2, num_neg_samples]
print("negative examples:", neg_edge_index)

shape of neg_edge_index: torch.Size([2, 5])
negative examples: tensor([[1560, 1554, 1187, 2477,  446],
        [2322, 2507, 2211,  363, 2309]])


Positive examples (`edge_label_index`) will be assigned the label 1, and negative ones will be assigned the label 0. We can obtain the label of positive examples like this:

In [119]:
print("positive examples' labels:", train_data.edge_label)

positive examples' labels: tensor([1., 1., 1.,  ..., 1., 1., 1.])


Now we can construct training and testing pipeline, which is similar to what we do in the last Colab. 

#### Question 2

Please follow the instruction and implement a function that trains a model.

In [141]:
def train(model, data, optimizer, loss_fn):

    loss = 0

    # TODO: Define train function.
    # 1. put the model into train mode
    # 2. clear the gradients calculated from the last batch
    # 3. get the node representation by model
    # 4. sample the negative examples with the same number of positive ones (edge_label_index)
    # 5. concatenate the positive edges and negative edges
    # 6. concatenate the labels of positive edges and negative edges
    # 7. calculate the similarity between two end nodes to determine the probability that the corresponding edge is present on the graph.
    # 8. feed the probability and edge label to the loss function
    # 9. calculate the gradients of each parameter
    # 10. update the parameters by taking an optimizer step

    ############# Your code here ############
    ## (~10 line of code)

    model.train()
    optimizer.zero_grad()
    z = model(data.x, data.edge_index)

    neg_edge_index = negative_sampling(
        edge_index=train_data.edge_index, num_nodes=train_data.num_nodes,
        num_neg_samples=train_data.edge_label_index.size(1))

    edge_label_index = torch.cat(
        [train_data.edge_label_index, neg_edge_index],
        dim=-1,
    )
    edge_label = torch.cat([
        train_data.edge_label,
        train_data.edge_label.new_zeros(neg_edge_index.size(1))
    ], dim=0)

    out = compute_similarity(z, edge_label_index).view(-1)
    loss = loss_fn(out, edge_label)
    loss.backward()
    optimizer.step()

    #########################################

    return loss

We usually use [AUC score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html) to evaluate the performance of model on binary classification task. The test function is as followed:

In [142]:
from sklearn.metrics import roc_auc_score

@torch.no_grad()
def test(model, data):
    model.eval()
    z = model(data.x, data.edge_index)
    out = compute_similarity(z, data.edge_label_index).view(-1).sigmoid()
    return roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())

Now we can start to train our model based on `train` and `test` function:

In [ ]:
epochs = 50

best_val_auc = final_test_auc = 0
for epoch in range(1, epochs + 1):
    loss = train(model, train_data, optimizer, loss_fn)
    valid_auc = test(model, val_data)
    test_auc = test(model, test_data)
    if valid_auc > best_val_auc:
        best_val_auc = valid_auc
        final_test_auc = test_auc
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {valid_auc:.4f}, Test: {test_auc:.4f}')

## Submission

Make sure to run all the cells and save a copy of this colab in your driver. If you complete this notebook, download the colab and upload your work to canvas to submit it.